# Development of Tool Calling functionality and py module - OpenAI

> The code developed here is finally copied to `llm/controller/tool_calling.py` and available as the `llm.controller.tool_calling` module

This notebook develops the tool-calling functionality on top of plain one-shot prompting. While OpenAI is chosen as the LLM provider, the code should work for most everyone with hopefully minor modifcations.

While `tool` and `using tools in LLM interactions` belong together, I am splitting the development into two notebooks since a single notbook covering both aspects was getting too large.
 - The [tool infrastructure](./Py_mod_llm_tools_devel.ipynb) that helps define the tools, generating their schemas and such.
 - This notebook which details the higher-level loop/driver _(shown below)_ that takes in a bunch of tool definitions and drives a chat to completion.

 ![](../img/tool_calling_protocol.png)

The hello-world of tool-calling, _get_weather_ is described in the official [OpenAI docs](https://platform.openai.com/docs/guides/function-calling). The OpenAI implementation _(atleast when it came out in late 2023)_ did not specify how one would build the json spec.Our implementation here demonstrates the use of pydantic classes to
 - Automate generation of the JSON schema required for tools
 - Automate deserialization of the JSON args supplied by OpenAI
 - Simplify implementation of tooling for ReAct and any other LLM use case.
 - Similar infra is also used for obtaining structured outputs from OpenAI and others.

![](../img/tool_calling_protocol_impl_details.png)

There are two scenarios demonstrated here
 - `What is the weather in XX` which validates basic tool calling
 - `Increase Temperature by YY` which tests if the LLM can first call `get_temperature`, do some math and then call `set_temperature`.

## Setup Basic Environment

> Normally, these would go into a python module and I would include it in my 
module search path. However, when executed in colab directly from github, It requires that I put my lib code in a repo, clone said repo into colab environment and then add it to my path. Will deal with that later!

 - Logging
 - Colab environment
 - OpenAI functions 

In [1]:
# Allow Colab or VSCore/Normal Jupyter environments
import os
from pathlib import Path

# The default relative path when running the notebook from a cloned repo.
LIB_PATH = Path("../lib")

if 'google.colab' in str(get_ipython()):
    print("👉 Setting up for Colab")
    # This will create a py-llm dir at the same level as this notebook
    # Refer to the lib in there using `./py-llm/lib` as opposed to the 
    # relative `../lib` when we are running straight from the py-llm/nbs 
    # directory in VScode.
    if not os.path.isdir("./py-llm"):
        print("Cloning git repo into ./py-llm")
        !git clone -b 3_llm_tools_and_support https://github.com/vamsi-juvvi/py-llm.git
        LIB_PATH = Path("./py-llm/lib")
    else:
        print("./py-llm exists. Not cloning") 

In [14]:
# Append to sys.path directly
# Make sure to `str(Path)`
# - The resolve() converts relative to absolute. 
# - If you use ~ for HOME, use `Path.expand_user()`
import sys
import logging

sys.path.append(str(LIB_PATH.resolve()))

from py_llm.util import jupyter_util
from py_llm.util.jupyter_util import TextAlign
from py_llm.util.jupyter_util import DisplayHTML as DH
from py_llm.util.jupyter_util import DisplayMarkdown as DM
from py_llm.util.jupyter_util import ColabEnv

from py_llm.llm import openai_util as oai
from py_llm.llm.tools import Tool, ToolCollection

# Init jupyter env
jupyter_util.setup_logging(logging.DEBUG)

In [12]:
# Uncomment for use in Colab or when the package is missing
#!pip install -qy openai

In [13]:
import openai

# If you want to log OpenAI's python library itself, also set the log level for this
# normally, limit this to warning/error and keep your own logging at debug levels.
# If this doesn't work right away, restart the kernel after changing the log-level
os.environ["OPENAI_LOG"]="error"

# Finally ensure you have the OpenAI key.
openai.api_key = ColabEnv.colab_keyval_or_env("OPENAI_API_KEY")
assert(openai.api_key)

## Use llm.tools module for our tools

↪ The development notes for tool-schema creation etc have been moved to [Py_mod_llm_tools_devel.ipynb](./Py_mod_llm_tools_devel.ipynb) where further evolution notes are also maintained. The code from that notebook was copied into the `llm.tools` module and that is what will be used in the rest of this notebook.

## The chat loop with tools involved

Unlike a one-shot prompt, when tools are used:

 - the LLM can respond with one or more `tool call`s instead of an `assistant response`. 
 - We need to evaluate all the tool calls and respond. 
 - This is continued till the LLM responsds with an assistant response 
 - Then we are done.

![](../img/toolcollection_calling_protocol_impl_details.png) 

In [23]:
def run_chat_loop(prompt:str, tools : ToolCollection):
    """
    Runs a chat loop with an initial prompt and supplied tools
    Resolves all tool_calls made till a final assistant response is provided
    """
    # Initialize
    chat_history = [
        {
            "role" : "system",
            "content" : "You are a helpful assistant that uses the supplied tools to respond to the user's questions."
        }]

    tool_schemas = tools.get_schemas()

    # Run the loop
    msgs = [{
        "role":"user", 
        "content": prompt}]

    DH.color_box(prompt, title="Prompt", bg="yellow")
    
    while len(msgs):
        chat_history.extend(msgs)
        msgs = []

        response = oai.get_response(
            chat_history=chat_history,
            tools = tool_schemas)

        # tool-call
        # Note: The OpenAI example is outdated
        # tool_calls is not longer a JSON object but an array of 
        # `ChatCompletionMessageToolCall` objects
        if response.choices[0].message.tool_calls:

            # The tool-call set needs to be added back to the chat_history
            msgs.append(response.choices[0].message)

            # Process all the tool calls
            for tool_call in response.choices[0].message.tool_calls:
                logging.debug(f"Executing tool_call: {tool_call}")            

                DH.color_box(f"{tool_call.function.name}({tool_call.function.arguments})", 
                             title="LLM Tool Call",
                             bg="lightgreen",
                             align = TextAlign.RIGHT)
                tool_result = tools.exec_tool(
                    tool_call.function.name,
                    tool_call.function.arguments)
                assert(isinstance(tool_result, str))

                # along with it's response. The response will be linked to the tool_call's 
                # via the ID        
                msgs.append({
                    "role" : "tool",
                    "tool_call_id" : tool_call.id,
                    "content"      : tool_result
                })
        else:
            # Assistant response
            chat_response = response.choices[0].message.content
            DH.color_box(chat_response, title="Final LLM Response", align=TextAlign.LEFT)

## Implement OpenAI's get_weather example

See https://platform.openai.com/docs/guides/function-calling. I an including it inline below

----

```python
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)
```

My goal is to automate the creation of the json _(the `tools` variable above)_ given a function. The previously created `getToolJsonSchema` handles the creation of the schema given a function: the imposed limitation is that the function, if it has arguments, is limited to just 1 and it must be a pydantic class.

> 👉 Note that in a production scenario, one will enfore that all the fields and the function itself will have a reasonable descriptions. We need good natural language descriptions to allow the LLM to decide which tool to call.

### Create a datamodel for weather args and a function that uses it

In [20]:
from pydantic import BaseModel, Field
from dataclasses import dataclass

@dataclass
class GetWeather(BaseModel):        
    location : str = Field(description="City and country e.g. San Jose, USA")

def get_weather(args: GetWeather) -> float:
    """
    Get current temperature for a given location.
    """
    retval = "10"
    logging.debug(f"get_weather called with {args}. Returning hardcoded value {retval}")
    return retval

# Register in the tool dictionary
# Note that one could evolve this further to make the tool_dict the single 
# source and use it to the tool descriptions as well.
gw_tools = ToolCollection()
gw_tools.register_tool(Tool(get_weather))

10:54:45 DEBUG:Tool : get_weather, Initialization
10:54:45 DEBUG:Tool : get_weather, Schema=
{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "strict": true,
        "parameters": {
            "properties": {
                "location": {
                    "description": "City and country e.g. San Jose, USA",
                    "title": "Location",
                    "type": "string"
                }
            },
            "required": [
                "location"
            ],
            "title": "GetWeather",
            "type": "object",
            "additionalProperties": false
        }
    }
}



In [21]:
# Test schema generation and execution!
if 1:
        DM.h("Test get_weather call manually", title_level=2)

        # Verify that the JSON we get from out `get_weather` function matches the 
        # raw JSON used in the OpenAI example
        DM.h("The autogenerated schema for `get_weather`", title_level=3)
        DM.json(gw_tools.get_schemas())

        # Test using the OpenAI example's serialized JSON
        DM.h("Executing call throught ToolCollection API", title_level=3)
        DM.code(gw_tools.exec_tool("get_weather", "{\"location\":\"Paris, France\"}"))                

## Test get_weather call manually

### The autogenerated schema for `get_weather`

```json
[
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for a given location.",
            "strict": true,
            "parameters": {
                "properties": {
                    "location": {
                        "description": "City and country e.g. San Jose, USA",
                        "title": "Location",
                        "type": "string"
                    }
                },
                "required": [
                    "location"
                ],
                "title": "GetWeather",
                "type": "object",
                "additionalProperties": false
            }
        }
    }
]
```

### Executing call throught ToolCollection API

10:54:46 DEBUG:Executing tool: get_weather
10:54:46 DEBUG:Attempting to deserialize {"location":"Paris, France"} for tool: get_weather
10:54:46 DEBUG:✔️ deserialized to location='Paris, France'. Calling function
10:54:46 DEBUG:get_weather called with location='Paris, France'. Returning hardcoded value 10
10:54:46 DEBUG:✔️ function returned 10


```None
10
```

In [24]:
if 1:
        DM.h("Test get_weather via run_chat_loop ⇔ LLM ", title_level=2)        

        run_chat_loop(
            prompt="What is the weather in San Jose, USA?",
            tools = gw_tools
        )

## Test get_weather via run_chat_loop ⇔ LLM 

10:55:23 DEBUG:Executing tool_call: ChatCompletionMessageToolCall(id='call_LHz0TAGCtkhyVhnzxd4EfG49', function=Function(arguments='{"location":"San Jose, USA"}', name='get_weather'), type='function')


10:55:23 DEBUG:Executing tool: get_weather
10:55:23 DEBUG:Attempting to deserialize {"location":"San Jose, USA"} for tool: get_weather
10:55:23 DEBUG:✔️ deserialized to location='San Jose, USA'. Calling function
10:55:23 DEBUG:get_weather called with location='San Jose, USA'. Returning hardcoded value 10
10:55:23 DEBUG:✔️ function returned 10


In [27]:
@dataclass
class SetTemperature(BaseModel):        
    temp : float = Field(description="The temperature value in Fahrenheit to set the thermostat to")

def set_thermostat_temperature(args: SetTemperature) -> str:
    """
    Sets the current temperature for a given location."
    """
    logging.debug(f"set_thermostat_temperature called with {args}")
    return ""

def get_thermostat_temperature() -> str:
    """
    Returns the current temperature setting of the thermostat."
    """
    retval = "60"
    logging.debug(f"get_thermostat_temperature called. Returning hardcoded {retval}")
    return retval

# Register in the tool dictionary
iot_tools = ToolCollection()
iot_tools.register_tool(Tool(set_thermostat_temperature))
iot_tools.register_tool(Tool(get_thermostat_temperature))

10:56:46 DEBUG:Tool : set_thermostat_temperature, Initialization
10:56:46 DEBUG:Tool : set_thermostat_temperature, Schema=
{
    "type": "function",
    "function": {
        "name": "set_thermostat_temperature",
        "description": "Sets the current temperature for a given location.\"",
        "strict": true,
        "parameters": {
            "properties": {
                "temp": {
                    "description": "The temperature value in Fahrenheit to set the thermostat to",
                    "title": "Temp",
                    "type": "number"
                }
            },
            "required": [
                "temp"
            ],
            "title": "SetTemperature",
            "type": "object",
            "additionalProperties": false
        }
    }
}

10:56:46 DEBUG:Tool : get_thermostat_temperature, Initialization
10:56:46 DEBUG:Tool : get_thermostat_temperature, Schema=
{
    "type": "function",
    "function": {
        "name": "get_thermostat_tempera

In [28]:
run_chat_loop(
    prompt="Increase the temperature by 10 degrees",
    tools = iot_tools
)

10:56:50 DEBUG:Executing tool_call: ChatCompletionMessageToolCall(id='call_oRYvzGUQOQU5NGHmh2e6AlTf', function=Function(arguments='{}', name='get_thermostat_temperature'), type='function')


10:56:50 DEBUG:Executing tool: get_thermostat_temperature
10:56:50 DEBUG:Calling no-arg function: get_thermostat_temperature
10:56:50 DEBUG:get_thermostat_temperature called. Returning hardcoded 60
10:56:50 DEBUG:✔️ function returned 60
10:56:50 DEBUG:Executing tool_call: ChatCompletionMessageToolCall(id='call_u7GPkqIg03vpOWzBy97xtifd', function=Function(arguments='{"temp":70}', name='set_thermostat_temperature'), type='function')


10:56:50 DEBUG:Executing tool: set_thermostat_temperature
10:56:50 DEBUG:Attempting to deserialize {"temp":70} for tool: set_thermostat_temperature
10:56:50 DEBUG:✔️ deserialized to temp=70.0. Calling function
10:56:50 DEBUG:set_thermostat_temperature called with temp=70.0
10:56:50 DEBUG:✔️ function returned 
